In [4]:
%cd "/home/dutta/Downloads/Summer'24/MIST_codes/scripts"
import read_mist_models

/home/dutta/Downloads/Summer'24/MIST_codes/scripts


In [5]:
%cd "/home/dutta/Downloads/MIST_ET"

/home/dutta/Downloads/MIST_ET


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
from astropy.table import vstack
import os
import glob
# import read_mist_models
from scipy.spatial import cKDTree

# Query template for GAIA DR3
query_template = """
SELECT
    source_id,
    parallax,
    ra,
    dec,
    phot_g_mean_mag + 5 * LOG10(parallax) - 10 AS mg,
    phot_bp_mean_mag AS mbp,
    phot_rp_mean_mag AS mrp,
    bp_rp,
    phot_g_mean_flux,
    phot_bp_mean_flux,
    phot_rp_mean_flux,
    ag_gspphot,
    ebpminrp_gspphot,
    pmra,
    pmdec
FROM
    gaiadr3.gaia_source
WHERE
    parallax_over_error > 10
    AND phot_bp_rp_excess_factor < 1.3 + 0.06 * POWER(bp_rp, 2)
    AND phot_bp_rp_excess_factor > 1.0 + 0.015 * POWER(bp_rp, 2)
    AND bp_rp IS NOT NULL
    AND phot_bp_mean_mag IS NOT NULL
    AND phot_rp_mean_mag IS NOT NULL
    AND ra BETWEEN {ra_min} AND {ra_max}
    AND (bp_rp - ebpminrp_gspphot) BETWEEN {bp_rp_min} AND {bp_rp_max}
    AND (phot_g_mean_mag + 5 * LOG10(parallax) - 10 - ag_gspphot) BETWEEN {g_min} AND {g_max}
"""

# Function to execute a query for a given RA range
def execute_query(ra_min, ra_max, bp_rp_min, bp_rp_max, g_min, g_max):
    query = query_template.format(
        ra_min=ra_min, ra_max=ra_max,
        bp_rp_min=bp_rp_min, bp_rp_max=bp_rp_max,
        g_min=g_min, g_max=g_max
    )
    job = Gaia.launch_job_async(query)
    return job.get_results()

# Define RA ranges to split the query
ra_ranges = [(0, 60), (60, 120), (120, 180), (180, 240), (240, 300), (300, 360)]

# Define bp_rp and g magnitude limitations
bp_rp_min = 0.0  # Update this value based on your criteria
bp_rp_max = 4.0  # Update this value based on your criteria
g_min = -8.0  # Update this value based on your criteria
g_max = -2.0  # Update this value based on your criteria

# Execute queries and combine results
results = []
for ra_min, ra_max in ra_ranges:
    result = execute_query(ra_min, ra_max, bp_rp_min, bp_rp_max, g_min, g_max)
    results.append(result)

# Combine all results into a single table
gaia_results = vstack(results).to_pandas()

# Extract GAIA data
gaia_results['bp_rp'] = gaia_results['bp_rp'] - gaia_results['ebpminrp_gspphot']
gaia_results['mg'] = gaia_results['mg'] - gaia_results['ag_gspphot']

In [ ]:
from scipy.spatial.distance import cdist

# Read synthetic star data from .eep.cmd files
def read_synthetic_data(eep_cmd_files):
    data = []
    for eep_cmd_file in eep_cmd_files:
        eepcmd = read_mist_models.EEPCMD(eep_cmd_file)
        B = eepcmd.eepcmds['Gaia_BP_EDR3']
        R = eepcmd.eepcmds['Gaia_RP_EDR3']
        G = eepcmd.eepcmds['Gaia_G_EDR3']
        A = eepcmd.eepcmds['star_age']
        P = eepcmd.eepcmds['phase']

        mask = (P == 6)
        A_filtered = A[mask]
        B_filtered = B[mask]
        R_filtered = R[mask]
        G_filtered = G[mask]
        bp_rp = B_filtered - R_filtered
        
        for i in range(len(A_filtered)):
            data.append([A_filtered[i], bp_rp[i], G_filtered[i], eep_cmd_file])
    return pd.DataFrame(data, columns=['star_age', 'bp_rp', 'G', 'file'])

# Change directory to the folder containing .eep.cmd files
os.chdir('.')  # Update this to your directory path

# Retrieve and sort .eep.cmd files
eep_cmd_files = glob.glob('*.eep.cmd')
eep_cmd_files.sort()  # Sort files by filename
eep_cmd_files = eep_cmd_files[:40]  # Consider only the first 40 files

# Read the data from the .eep.cmd files
df = read_synthetic_data(eep_cmd_files)

# Function to calculate distances and filter GAIA stars
def filter_gaia_stars(gaia_results, df, threshold=0.05):
    gaia_points = np.array(gaia_results[['bp_rp', 'mg']])
    synthetic_points = np.array(df[['bp_rp', 'G']])
    
    distances = cdist(synthetic_points, gaia_points, metric='euclidean')
    
    indices = np.where(distances < threshold)[1]
    
    return gaia_results.iloc[np.unique(indices)]

# Plot setup
plt.figure(figsize=(20, 17))
# plot_lines()

# Filter GAIA stars based on proximity to synthetic stars
gaia_results_line = gaia_results
selected_gaia_stars = filter_gaia_stars(gaia_results_line, df)

# Initialize a list to store the DataFrames
all_selected_gaia_stars = []

# Iterate through eep.cmd files
for idx, eep_cmd_file in enumerate(eep_cmd_files):
    file_data = df[df['file'] == eep_cmd_file].sort_values('star_age')
    
    # fastest_region_pre = find_fastest_regions(file_data)
    fastest_region_pre = file_data
    selected_gaia_stars = filter_gaia_stars(gaia_results_line, fastest_region_pre)
    fastest_region = fastest_region_pre
    
    if not fastest_region.empty:
        start_age = fastest_region['star_age'].iloc[0]
        end_age = fastest_region['star_age'].iloc[-1]
        mass = int(eep_cmd_file.split('M')[0]) / 1e4  # Extract and format mass
        
        plt.scatter(fastest_region['bp_rp'], fastest_region['G'], s=4, alpha=0.7, color='green')
        mid_idx = len(fastest_region) // 2
        plt.text(fastest_region['bp_rp'].iloc[mid_idx], fastest_region['G'].iloc[mid_idx], f"{mass}M", fontsize=8, color='black')
        diff = end_age - start_age
        plt.text(fastest_region['bp_rp'].iloc[0], fastest_region['G'].iloc[0], f"{diff:.1f}", fontsize=8, color='black')
        
        # Filter and plot GAIA stars for this eep.cmd file
        plt.scatter(selected_gaia_stars['bp_rp'], selected_gaia_stars['mg'], s=2, alpha=0.7, c='blue')
        print(selected_gaia_stars)
        
    # Append the DataFrame to the list
    all_selected_gaia_stars.append(selected_gaia_stars)

# Finalize plot
plt.xlabel('bp_rp')
plt.ylabel('G')
plt.gca().invert_yaxis()
plt.legend()
plt.savefig('selected_gaia_H_gap.png')
plt.show()

# Concatenate all DataFrames in the list
concatenated_gaia_stars = pd.concat(all_selected_gaia_stars, ignore_index=True)

# Save the concatenated DataFrame to a .txt file
concatenated_gaia_stars.to_csv('selected_gaia_stars.txt', sep='\t', index=False)

print("All selected GAIA stars have been saved to selected_gaia_stars.txt")

In [ ]:
all_selected_gaia_stars